In [65]:
import os

PROJECT_ID = "swift-kiln-380818"
LOCATION = "us-central1"

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/aditygrg/.config/gcloud/application_default_credentials.json"

import vertexai
from vertexai.generative_models import GenerativeModel, ChatSession

vertexai.init(location=LOCATION, project=PROJECT_ID)

In [78]:
import requests
from vertexai.generative_models import (
    Content,
    FunctionDeclaration,
    GenerationConfig,
    GenerativeModel,
    Part,
    Tool,
)
from vertexai.preview import rag

In [67]:
user_verified = FunctionDeclaration(
    name="user_verified",
    description="This is called when the user is successfully verified",
    parameters={
        "type": "object",
        "properties": {"status": {"type": "boolean"}},
    },
)

get_user_data = FunctionDeclaration(
    name="get_user_data",
    description="Get the info of the user to verify",
    parameters={
        "type": "object",
        "properties": {"phone_number": {"type": "string"}},
    },
)

user_not_verified = FunctionDeclaration(
    name="user_not_verified",
    description="This is called when user is not verified",
    parameters={
        "type": "object",
        "properties": {"status": {"type": "boolean"}},
    },
)

In [68]:
retail_tool = Tool(
    function_declarations=[
        get_user_data,
        user_verified,
        user_not_verified
    ],
)

In [69]:
SYSTEM_INSTRUCTION = """
You are an Amazon Customer Support Agent and you are compassionate and assuring to help your user always!

You need to always verify if you are talking to the same person as the account holder for security purposes.

If there is no user_data, you can ask for his phone number and input it to the `find_data` function to get the data.

After you have the data, you can verify then by asking pincode and city + state of the customer and matching it with the data.

The data you have should not be shared to the customer then it will be cheating.

Only move ahead with query once you are able to verify pincode + address of the customer.

Always start with: "Welcome to Amazon! and a good greeting"
"""

In [70]:
model = GenerativeModel(
    "gemini-1.5-pro-001",
    generation_config=GenerationConfig(temperature=0.3),
    tools=[retail_tool],
    system_instruction=SYSTEM_INSTRUCTION
)
chat = model.start_chat()

In [71]:
prompt = """
%User Query:
I am unable to order a product


%user_data: 
phone_no: 932489237
name: "Raj Patel"
email: "raja23@gmail.com"
subscriptionStatus: false
apartment_no "223, Suryasthali Appartment"
area_street "Manavta Nagar"
landmark "Hanuman Mandir"
town_city "Bengaluru"
state "Karnataka"
pincode 530068
"""

response = chat.send_message(prompt)
response.candidates[0].content.parts[0]

text: "Welcome to Amazon! I hope you are having a good day. I understand you are unable to order a product. Let me quickly verify your information and I will be right with you. Could you please confirm your pincode? \n"

In [75]:
messages = [
    ("AI", "Welcome to Amazon! I hope you are having a good day. I understand you are unable to order a product. Let me quickly verify your information and I will be right with you. Could you please confirm your pincode?"),
    ("Human", "Sure, my pincode is 530068"),
    ("AI", "Great! Could you also please confirm the city and state you are residing in? \n"),
    ("Human", "I am living in Bangalore, Karnataka")
]

In [76]:
def convert_to_str(messages):
    ans = ""
    for i in messages:
        ans += i[0] + " : " + i[1] + "\n"

    return ans

In [77]:
chat.send_message(convert_to_str(messages))

candidates {
  content {
    role: "model"
    parts {
      function_call {
        name: "user_verified"
        args {
          fields {
            key: "status"
            value {
              bool_value: true
            }
          }
        }
      }
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
    probability_score: 0.141162992
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.103205055
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
    probability_score: 0.488420635
    severity: HARM_SEVERITY_LOW
    severity_score: 0.300437599
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
    probability_score: 0.120542295
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.125065371
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
    probability_score: 0

In [58]:
response = chat.send_message(
    Part.from_function_response(
        name="user_not_verified",
        response={
            "content": "Sorry, we were unable to verify your details. Thank you for contacting Amazon!",
        },
    ),
)
response.candidates[0].content.parts[0]

text: "Sorry, we were unable to verify your details. Thank you for contacting Amazon! \n"

In [ ]:
prompt = """
What about the Pixel 8? Is there a store in
Mountain View, CA that I can visit to try one out?
"""

response = chat.send_message(prompt)
response.candidates[0].content.parts[0]

In [ ]:
# Here you can use your preferred method to make an API request and get a response.
# In this example, we'll use synthetic data to simulate a payload from an external API response.

api_response = {"store": "2000 N Shoreline Blvd, Mountain View, CA 94043, US"}

In [ ]:
response = chat.send_message(
    Part.from_function_response(
        name="get_store_location",
        response={
            "content": api_response,
        },
    ),
)
response.candidates[0].content.parts[0]

In [ ]:
api_response = {"sku": "GA08475-US", "in_stock": "yes"}


In [ ]:
response = chat.send_message(
    Part.from_function_response(
        name="get_product_info",
        response={
            "content": api_response,
        },
    ),
)
response.candidates[0].content.parts[0]

In [ ]:
prompt = """
I'd like to order a Pixel 8 Pro and have it shipped to 1155 Borregas Ave, Sunnyvale, CA 94089.
"""

response = chat.send_message(prompt)
response.candidates[0].content.parts[0]

In [ ]:
api_response = {
    "payment_status": "paid",
    "order_number": 12345,
    "est_arrival": "2 days",
}

In [ ]:
response = chat.send_message(
    Part.from_function_response(
        name="place_order",
        response={
            "content": api_response,
        },
    ),
)
response.candidates[0].content.parts[0]